# Philip


In [1]:
import numpy as np
from collections import Counter
import re
import pandas as pd

Cleaning function taken from D Greenberg's answer on: https://stackoverflow.com/questions/4576077/python-split-text-on-sentences

In [3]:
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    with open(str(text), "rb") as f:
        text = f.read().decode("UTF-8-sig").replace('\n', '').replace('\r', '')
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip().replace('’',"'").lower() for s in sentences]
    sentences = [s[0:-1] for s in sentences]
    return sentences

content=split_into_sentences('catcorpus.txt')

In [4]:
sentences=[] 
for line in content:
    token=line.replace(',','').split()
    sentences.append(token)
tokens = [j for i in sentences for j in i]

uniquetokens = list(set(tokens))

In [5]:
def ngrams(sentences):
    grams=[]
    global n
    n=int(input("Ngram, N = "))
    for sentence in sentences:
        gram=list(zip(*[sentence[g:] for g in range(n)]))
        grams.append(gram)
    grams=[j for i in grams for j in i]
    gram_counts=Counter(grams)
    grams_freq=gram_counts.most_common()
    grams_freq=[(' '.join(i[0][0:n-1]), i[0][n-1], i[1]) for i in grams_freq]
    grams_sorted=sorted(grams_freq)    
    return grams_sorted                       

In [8]:
ngram=ngrams(sentences)


columns=sorted(set([i[1] for i in ngram]))
rows=sorted(set([i[0] for i in ngram]))
columndict = dict(zip(columns, range(0,len(columns))))
rowdict = dict(zip(rows, range(0,len(rows))))
ngramM = np.zeros((len(rows),len(columns)),dtype=int)
for g in ngram:
    ngramM[rowdict[g[0]]][columndict[g[1]]]=g[2]
totals=np.sum(ngramM, axis=1, dtype=int)
dfngram=pd.DataFrame(ngramM)
dfngram.columns=[columns]
dfngram.index=[i for i in rows]
dfngram['Totals:']=totals
dfngram

Ngram, N = 2


,a,and,are,bugs,cat,cat's,does,eat,food,in,...,more,not,owns,rat,rats,tent,that,the,too,Totals:
a,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
and,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
are,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
bugs,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
cat,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,7
cat's,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,2
cats,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
does,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
eat,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
in,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [10]:
ngram_matrix=pd.DataFrame(dfngram).values
for sentence in content:
    print(sentence)
    
while True:
    print()
    given=input("Starting word or phrase (Words="+str(n-1)+"): ")
    if given in rowdict or given in columndict:
        break
    else:
        print()
        print("Word/Phrase is not in the document")
        continue
print()
vec_total=ngram_matrix[:,-1]
NM = ngram_matrix[:,0:-1]
pmatrix = NM/ vec_total[:,None]
gensent = [given]
try:
    for word in range(7):
        indmax=np.argmax(pmatrix[rowdict[given]])
        if n>2:
            nxt=list(columndict.keys())[list(columndict.values()).index(indmax)]
            given=given.split()
            given=given[1:]
            given=' '.join(word for word in given)
            given=given+' '+nxt
            gensent.append(nxt)
        else:
            given=list(columndict.keys())[list(columndict.values()).index(indmax)]
            gensent.append(given)
            
    gen_sentence='Generated sentence: '+' '.join(i for i in gensent)
    print(gen_sentence)
except KeyError:
    gen_sentence='Generated sentence: '+' '.join(i for i in gensent)
    print(gen_sentence)
    print()
    print("...No further expected words.")

this is a cat that lisa owns
this is cat's food
the cat likes cat's food
the cat likes rats too
the cat likes rats more
rats eat bugs
the cat does not like bugs
the cat kills the rat
the cat likes it
cats, rats, and bugs are in the tent

Starting word or phrase (Words=1): bugs

Generated sentence: bugs are in the cat likes rats and
